# 참고 링크
https://docs.ultralytics.com/ko/yolov5/tutorials/train_custom_data/#4-train

https://docs.ultralytics.com/ko/quickstart/#understanding-settings

https://www.youtube.com/watch?v=Ped-JdFEYdk&t=861s


---



---



드라이브에서 파일 가져오기

> 본래 단계에서는 생략

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!unzip -qq '/content/drive/MyDrive/dataset.zip' -d '/content/'

---

---



데이터 전처리

> 기존 JSON 파일에서 TXT 파일로 변환하는 과정

In [ ]:
import os
import json

path = "/content/labels/" #json2txt.ipynb 파일과 동일한 경로에 있기 때문에 다음과 같이 선언

file_lists = os.listdir(path) #path 경로 내 모든 파일명 리스트 형태로 불러오기
print(file_lists)

['NegativeDB_구름_강원도강릉시난곡동_NP020001_001_없음_00003960.json', 'NegativeDB_구름_강원도강릉시난곡동_NP020001_001_없음_00002640.json', 'NegativeDB_구름_강원도강릉시난곡동_NP020001_001_없음_00002580.json', 'NegativeDB_구름_강원도강릉시난곡동_NP020001_001_없음_00008580.json', 'NegativeDB_구름_강원도강릉시난곡동_NP020001_001_없음_00006120.json', 'NegativeDB_구름_강원도강릉시난곡동_NP020001_001_없음_00002280.json', 'NegativeDB_구름_강원도강릉시난곡동_NP020001_001_없음_00002340.json', 'NegativeDB_구름_강원도강릉시난곡동_NP020001_001_없음_00003000.json', 'NegativeDB_구름_강원도강릉시난곡동_NP020001_001_없음_00004680.json', 'NegativeDB_구름_강원도강릉시난곡동_NP020001_001_없음_00002040.json', 'NegativeDB_구름_강원도강릉시난곡동_NP020001_001_없음_00002880.json', 'NegativeDB_구름_강원도강릉시난곡동_NP020001_001_없음_00008640.json', 'NegativeDB_구름_강원도강릉시난곡동_NP020001_001_없음_00004560.json', 'NegativeDB_구름_강원도강릉시난곡동_NP020001_001_없음_00004740.json', 'NegativeDB_구름_강원도강릉시난곡동_NP020001_001_없음_00000660.json', 'NegativeDB_구름_강원도강릉시난곡동_NP020001_001_없음_00006900.json', 'NegativeDB_구름_강원도강릉시난곡동_NP020001_001_없음_00007200.json', 'NegativeDB_구름_강원도강릉시난곡동_NP020

In [ ]:
#.json 확장자만 저장하기
json_file_lists = []
for file_name in sorted(file_lists):
    if file_name.split(".")[-1] == "json":
        json_file_lists.append(file_name)

In [ ]:
for j, json_file in enumerate(json_file_lists):
  #print(j,  json_file) #00000005.json 한개에 대해서 json2txt 변환하기 위해 해당 인덱스에 00000005.json 파일 불러오기
  with open(os.path.join(path, json_file),"r") as file:
      json_data = json.load(file) #json 파일 내 데이터 읽어오기

  WIDTH = float(json_data['images'][0]['width'])
  HEIGHT = float(json_data['images'][0]['height'])

  new_file = open(os.path.join(path, json_data['images'][0]['file_name'].split(".")[0]+'.txt'),"w") #"w": 쓰기 형식, "r": 읽기 형식, "a": 이어쓰기 형식
  #print(os.path.join(path, json_data['images'][0]['file_name'].split(".")[0]+'.txt'))
  #CLASS_NAMES = ['흑색연기', '백색/회색연기', '화염', '구름', '안개/연무', '굴뚝연기'] # 파일 내 커스텀 클래스 정보

  #json 데이터 내 'shapes' 정보 읽어오기
  for i, data in enumerate(json_data['annotations']):
      #print("   ", i) #순차적으로 for문이 진행되는지 확인하는 인덱스 번호 출력
      class_id = data['category_id']
      if class_id<0 or class_id>6:
        print(class_id)
      x_left_top = float(data['bbox'][0]) #물체의 바운딩 박스, left top x좌표
      y_left_top = float(data['bbox'][1]) #물체의 바운딩 박스, left top y좌표
      x_right_bottom = float(data['bbox'][2]) #물체의 바운딩 박스, right bottom x좌표
      y_right_bottom = float(data['bbox'][3]) #물체의 바운딩 박스, right bottom y좌표
      width = x_right_bottom - x_left_top #물체의 바운딩 박스 너비
      height = y_right_bottom - y_left_top #물체의 바운딩 박스 높이
      normalize_width = abs(width) / WIDTH #YOLO 학습을 위해 너비 정규화
      normalize_height = abs(height) / HEIGHT #YOLO 학습을 위해 높이 정규화
      ctr_x = x_left_top + width / 2 #물체 바운딩 박스 중심 x 좌표
      normalize_ctr_x = ctr_x / WIDTH #YOLO 학습을 위해 물체 바운딩 박스 중심 x 좌표 정규화
      ctr_y = y_left_top + height / 2 #물체 바운딩 박스 중심 y 좌표
      normalize_ctr_y = ctr_y / HEIGHT #YOLO 학습을 위해 물체 바운딩 박스 중심 y 좌표 정규화
      #텍스트 파일에 YOLO 학습용 데이터 포맷으로 정리
      #모든 값은 string 타입으로 저장해야함
      new_data = "{} {} {} {} {}\n".format(str(class_id), str(normalize_ctr_x), str(normalize_ctr_y), str(normalize_width), str(normalize_height))

      new_file.write(new_data)

  new_file.close()